# Praten met LLMs

Iedereen weet hoe ze moeten praten met LLMs als ChatGPT, Gemini, Claude, etc.: via een website. Dat is voor individueel gebruik natuurlijk perfect, maar wat als je een LLM voor een eigen project in wil zetten? Dan kan dat niet via de browser, dit doen we met een API. Hieronder heb ik een voorbeeldje om te praten met een model van [Nebius Studio](https://tokenfactory.nebius.com/). Dit is een platform waar je gratis 1 euro krijgt om met LLMs te praten. Dit is waarschijnlijk niet genoeg voor je project, maar voor kleine tests werkt dat uitstekend.

Eerst de imports dan maar: Nebius werkt op de zelfde manier als Open AI, dit is een soort standaard geworden voor vele LLM API aanbieders

In [ ]:
# pip install openai

In [ ]:
import os
from openai import OpenAI
import numpy as np

Als we werken met een API hebben we een *key* nodig. Dit is een verificatiesleutel die je stuurt naar de API waarmee je laat weten dat jij het bent.

Ik heb op de nebius studio website een key aangemaakt en toegevoegd aan mijn windows systeem via de volgende stappen:
- Press Start -> type “Environment Variables” -> open “Edit the system environment variables”
- Click Environment Variables
- Under User variables (or System variables):
  - Click New
  - Variable name: HCAI_NEBIUS_API_KEY
  - Variable value: your key
- Press OK -> restart your notebook\
*(Steps generated with ChatGPT)*

<span style="color: red; font-weight: bold;">LET OP: Je wil NOOIT je API key delen. Hiermee kunnen anderen mensen jou geld uitgeven</span>   


<span style="color: red; font-weight: bold;">Push de key dus nooit naar GitHub. Ook niet als je deze ergens in een jupyter notebook hebt gebruikt. Dit valt allemaal terug te halen. Reset dan eerst je notebook en zet deze daarna pas in Git.</span>   

In [ ]:
NEBIUS_API_KEY = os.environ.get("HCAI_NEBIUS_API_KEY") # <- Ik heb mijn API key `HCAI_NEBIUS_API_KEY` genoemd, als die van jou anders heet, pas deze dan aan

if not NEBIUS_API_KEY:
    raise ValueError("NEBIUS_API_KEY is not set in environment variables.")

Nu kunnen we contact maken met de API. Hiervoor maken we een client aan. De client is onze verbinding met de API. We hoeven hier nog niet ons model te specificeren, dat komt straks pas.

In [ ]:
client = OpenAI(
    api_key=NEBIUS_API_KEY,
    base_url="https://api.studio.nebius.ai/v1",
)

Oke, nu kunnen we een functie gaan schrijven die ene vraag stelt aan het model en het antwoord teruggeeft. Hieronder een voorbeeld:

In [ ]:
def ask_nebius(model: str, prompt: str)-> tuple[object, str, str]:
    """
    Sends a user message to a Nebius Studio LLM and returns the model's reply.
    """
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt} 
        ],
    )
    response = client.chat.completions.create(
        model=model,
        messages=[ # Hier geven we aan hoe het model de prompt van de user moet ontvangen.
            # hier kunnen we een system prompt meegeven. 
            # Dit kan bijvoorbeeld een instructie zijn voor hoe het model zich moet gedragen tegenover de gebruiker.
            {
                "role": "system",
                "content": """SYSTEM_PROMPT""" 
            },
            # Hier geven we de user prompt.
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    )
    
    # De LLM geeft veel informatie terug, maar we zijn alleen geïnteresseerd in de response tekst. 
    # We geven eerst alle output, dan alle output als een json format, en daarna alleen de message response.
    return response, response.to_json(), response.choices[0].message.content

Oke, laten we een simpele vraag stellen:

In [ ]:
# Op dit moment is dit model het goedkoopst
model = "google/gemma-2-2b-it"
response, json_response, text_reply = ask_nebius(model, "Give me the python code for a hello world example.")

print("Model reply:\n", text_reply)


Hieronder heb ik de output die ik zelf kreeg geplakt in markdown, zodat de opmaak klopt:

---

Model reply:
 ```python
print("Hello, world!")
```

**Explanation:**

* **`print()`:** This is a built-in Python function that displays whatever you put inside its parentheses on the screen.
* **"Hello, world!"** : This is a string, a sequence of characters enclosed within quotation marks.  This is the message you want to display.


**To run this code:**

1. **Save it:** Create a new file named `hello_world.py` and paste this code into it.
2. **Open a terminal:** Navigate to the directory containing the file using the `cd` command.
3. **Run the code:** Type `python hello_world.py` and press Enter.


You'll see "Hello, world!" printed in your terminal. 



---
# Prompt tuning
Oke, je weet nu hoe je met een LLM moet praten enzo, maar hoe moet je nou de juiste prompt stellen aan ChatGPT? Dit is nog een best lastige vraag, zeker voor complexe vraagstukken. In dit stukje van dit notebook gaan we kijken naar het automatisch trainen van zo'n prompt. 

Het voorbeeld dat we gebruiken is het omdraaien van de volgorde van letters in een woord. Dus van "hoi" naar "ioh" bijvoorbeeld.

**NOTE:** We gaan hier niet de daadwerkelijke backpropagation bouwen. Dit stukje laat ik aan jullie over en hiervoor kan je prima een bestaande codebase gebruiken of ChatGPT om hulp voor vragen. Dit is complex en dit is iets dat je echt zelf uit zal moeten pluizen.

Om prompt tuning uit te voeren maken we gebruik van de `ask_nebius()` functie hierboven. Eerst definiëren we een kleine dataset:

In [ ]:
data = [
    ("cat", "tac"),
    ("hello", "olleh"),
    ("pizza", "azzip"),
]

Dan maken we nu een 'soft prompt', dat is de prompt die we gaan trainen.

Hier gebruiken we een hele kleine prompt, met 5 tokens, en 16-dimensionale embeddings.

Daarbij maken we een functie die de tokens omzet naar tekst die leesbaar is voor het model. In dit geval gebruiken we hier ook een dummy voorbeeld voor.

In [ ]:
soft_prompt = np.random.randn(5, 16)  

def render_soft_prompt(sp):
    return "<SP>" * len(sp)  # Fake textual stand‑in of `<SP>`

Nu kunnen we de training loop bouwen. We updaten de prompt voor 3 epochs, waarbij we een binaire loss definiëren, die allen kijkt of het klopt of niet.

In [ ]:
def loss(pred, target):
    return sum(a!=b for a,b in zip(pred, target))

n_epochs = 3
for epoch in range(n_epochs):
    total_loss = 0
    for x, y in data:
        sp_text = render_soft_prompt(soft_prompt)
        _, _, pred = ask_nebius(model, sp_text + " " + x)
        l = loss(pred, y)
        total_loss += l
        # Fake gradient update
        soft_prompt -= 0.01 * np.sign(np.random.randn(*soft_prompt.shape))
    print(f"Epoch {epoch} loss = {total_loss}")

Waarschijnlijk zien we hier niet dat het model wat geleerd heeft. Dit komt omdat we weinig epochs gebruiken, de gradient niet goed toepassen en deze niet eens echt gebruiken.

Toch gaan we even kijken of het heeft gewerkt, met een test set.

In [ ]:
test_words = ["apple", "melon"]

for w in test_words:
    sp_text = render_soft_prompt(soft_prompt)
    print(w, "->", ask_nebius("meta-llama/Meta-Llama-3.1-8B-Instruct", sp_text + " " + w)[2])

Oke, dit doet het niet, maar dat hadden we ook niet verwacht, dus dat is prima.

Wat ik wil laten zien is het idee achter prompt tuning. We geven het model een lijst met random noise als prompt en we gebruiken onze dataset om deze prompt beter en beter te maken, zodat het model de taak uit gaat voeren. 

Mocht je dit toe willen passen moet je dus op de juiste manier de gradients van je prompt updaten **en** de prompt tokens op de juiste manier meegeven aan het model.

---

# Finetuning met Nebius

Uiteraard kunnen we ook de LLM finetunen in plaats van alleen de prompt. Dit is echter een duurdere operatie, waardoor ik deze niet in dit notebook uit zal voeren. Echter heb ik een fantastische blogpost gevonden die laat zien hoe je voor ongeveer 10 euro LLAMA 3.1 op  de ToolACE dataset kan finetunen, waar 26.500 tekst datapunten in zitten.

🔗 https://nebius.com/blog/posts/fine-tuning-llms-with-nebius-ai-studio

Hiermee verhogen ze de accuracy van LLAMA 3.1 van 0.842 naar 0.899. Hoewel dat een significante verbetering is, gebruiken ze wel heel erg veel gelabelde data in. De kans dat jullie dit binnen jullie projecten ook hebben is niet zo groot. Waarschijnlijk ga je geen significante verbetering realiseren met een kleine dataset.

Daarbij zal je de getrainde LLM ergens moeten hosten. Hiervoor heb je ook een GPU nodig. Dit is altijd duurder dan het gebruik van voorgetrainde LLMs, omdat hier meerdere mensen tegelijk gebruik van kunnen maken, en van je eigen LLM maak je alleen zelf gebruik. Doe dit dus alleen als je een concreet plan hebt hoe en waar je de getrainde LLM gaat hosten en als je weet wat dit gaat kosten. Overleg dit met je opdrachtgever.

Tot slot durf ik niet te zeggen of dit data-vriendelijk is en wat Nebius met je data doet. Dit is aan jullie als HCAI studenten om uit te zoeken en te overleggen met de opdrachtgevers.

Kortom: deze techniek raad ik dus af.